<a href="https://colab.research.google.com/github/rahiakela/machine-learning-algorithms/blob/main/neural-networks-from-scratch/05-calculating-loss/cross_entropy_loss_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cross-Entropy Loss from Scratch

To train a model, we tweak the weights and biases to improve the model’s accuracy and confidence. To do this, we calculate how much error the model has. The loss function , also referred to as the cost function , is the algorithm that quantifies how wrong a model is. Loss is the measure of this metric. Since loss is the model’s error, we ideally want it to be 0.

The output of a neural network is actually confidence, and more confidence in the correct answer is better. Because of this, we strive to increase correct confidence and decrease misplaced confidence.

##Setup

In [ ]:
import numpy as np

## Categorical Cross-Entropy Loss

If you’re familiar with linear regression, then you already know one of the loss functions used with neural networks that do regression: squared error (or mean squared error with neural networks).

In [2]:
np.log(0.7)

-0.35667494393873245